In [1]:
!pip install pydantic

In [2]:
!pip install --upgrade boto3

In [3]:
!pip install --upgrade langchain

In [4]:
pip install langchain-community

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_community.llms import SagemakerEndpoint
from langchain.docstore.document import Document
from langchain.agents import load_tools, initialize_agent, Tool

from langchain.agents.react.base import DocstoreExplorer
from langchain_community.llms.sagemaker_endpoint import ContentHandlerBase, LLMContentHandler
from typing import Dict ,List
import json
#import Anthropic
from langchain.llms import Anthropic
from langchain.prompts import StringPromptTemplate
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser 

from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
from langchain.docstore.base import Docstore
import re

In [6]:
import boto3
boto3_session = boto3.Session(
    aws_access_key_id='xyz',
    aws_secret_access_key='xyz',
    region_name='us-east-1'
)
kendra_client = boto3_session.client('kendra')
kendra_client = boto3.client("kendra", region_name='us-east-1')

In [7]:
import os

os.environ.pop('AWS_ACCESS_KEY_ID', None)
os.environ.pop('AWS_SECRET_ACCESS_KEY', None)
os.environ.pop('AWS_SESSION_TOKEN', None)

In [8]:
!pip install --upgrade boto3 botocore

In [32]:
class Kendra(Docstore):
    """Kendra API Wrapper to search consulting documents."""

    def __init__(self, kendra_index_id: str, region_name: str) -> None:
        """Init Kendra client."""
        
        self.kendra_index_id = kendra_index_id
        
        try:
            import boto3
            self.kendra_client = boto3.client("kendra", region_name=region_name)
        except ImportError:
            raise ValueError(
                "Could not import boto3 python package"
            )

    def search(self, query: str) -> str:
        """Searching for document in Kendra Index and returning content."""
        try:
            result = self.kendra_client.retrieve(
                IndexId=self.kendra_index_id,
                QueryText=query,
                PageSize=2,
            )
            print("Debug: Full result object from Kendra:", result)
        except self.kendra_client.exceptions.ResourceNotFoundException as e:
            return "Kendra index not found: {}".format(e)
        except self.kendra_client.exceptions.ValidationException as e:
            return "Invalid query: {}".format(e)
        except Exception as e:
            return "An error occurred: {}".format(e)

        if not result["ResultItems"]:
            print("Debug: No ResultItems returned from Kendra query.")
            return "No relevant passages found."

        # adding all results together
        content = ""
        for item in result["ResultItems"]:
            title = item.get("DocumentTitle", "No title")
            uri = item.get("DocumentURI", "No URI")
            passage = item.get("Content", "No content")
            content += f"[Title: {title}, URI: {uri}, Passage content: {passage}] "
        
        print("Debug: Content retrieved from Kendra:", content)
        return content

kendra_docstore = Kendra(kendra_index_id="0044a5c2-b5fe-4ed1-a8a6-e025cf50d589", region_name='us-east-1')

In [35]:
result = kendra_docstore.search("tell me about assets in big companies and how they should be managed")
print(result)

Debug: Full result object from Kendra: {'QueryId': 'a61438ee-8f87-4556-8b6b-0db617b27a70', 'ResultItems': [{'Id': 'a61438ee-8f87-4556-8b6b-0db617b27a70-7c8f264e-e1ee-4fe7-bae6-be0dc973d6af', 'DocumentId': 's3://docs-consulting/documents/c2.pdf', 'DocumentTitle': 'c2', 'Content': 'While most are taking immediate action to regain balance, their focus is on the short term. To address the root of their problems, they should follow the counsel they would give their clients: Take a step back to reassess. Consulting firms should hone back in on the fundamentals, reset around the core principles of top-management consulting, and use transformative innovations, particularly AI, to refocus on their top-management product and service offerings. In so doing, they should also rethink the breadth and depth of those practices and support infrastructures. Staff behind demand. It’s time to take advantage of the retrenchment in staff numbers and greater client selectivity to staff their firms in respons

In [36]:
result = kendra_docstore.search("how technology is shifting the consulting landscape")
print(result)

Debug: Full result object from Kendra: {'QueryId': '999ed3d4-6b8d-4040-843c-43b7fe4eb021', 'ResultItems': [{'Id': '999ed3d4-6b8d-4040-843c-43b7fe4eb021-ae339eb5-3688-4dff-a4a1-74f4cac37824', 'DocumentId': 's3://docs-consulting/documents/c1.pdf', 'DocumentTitle': 'c1', 'Content': 'Purposes 1 through 5 are generally considered legitimate functions, though some controversy surrounds purpose 5. Management consultants are less likely to address purposes 6 through 8 explicitly, and their clients are not as likely to request them. But leading firms and their clients are beginning to approach lower-numbered purposes in ways that involve the other goals as well. Goals 6 through 8 are best considered by-products of earlier purposes, not additional objectives that become relevant only when the other purposes have been achieved. They are essential to effective consulting even if not recognized as explicit goals when the engagement begins. Moving up the pyramid toward more ambitious purposes requir

In [9]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"
    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        dialog = [{"role": "user", "content": prompt}]
        input_str = json.dumps({ "inputs" : [dialog], **model_kwargs})
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        model_predictions = json.loads(output.read())
        return model_predictions[0]['generation']['content']
content_handler = ContentHandler()

In [10]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor ,ReActChain
from langchain import  LLMChain

In [17]:
tools = [
    Tool(
        name="Search",
        func=kendra_docstore.search,
        description="useful for when you need to answer legal questions regarding contract ,regulations, policies"
    )
]

#endpoint_name="jumpstart-dft-meta-textgeneration-llama-2-70b-f",
llm = SagemakerEndpoint(
            endpoint_name="jumpstart-dft-meta-textgeneration-llama-2-70b-f",
            region_name="us-east-1",
            content_handler=content_handler,
            endpoint_kwargs={'CustomAttributes':'accept_eula=true'}
    )

In [22]:
class CustomOutputParser(AgentOutputParser):
    ai_prefix: str = "AI"
    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        if f"{self.ai_prefix}:" in text:
            return AgentFinish(
                {"output": text.split(f"{self.ai_prefix}:")[-1].strip()}, text
            )
        regex = r"Action: (.*?)[\n]*Action Input: ((.|\n)*)"
        match = re.search(regex, text)
        if not match:
            
            print("text :",text)
            raise OutputParserException(f"Could not parse LLM output: `{text}`")
        action = match.group(1)
        action_input = match.group(2)
        return AgentAction(action.strip(), action_input.strip(" ").strip('"'), text)

In [37]:
!pip install --upgrade langchain
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 8.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.7/906.7 kB 8.1 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.0/297.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.1/279.1 kB 21.9 MB/s eta 0:00:00


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.agents import LLMSingleActionAgent, AgentExecutor, Tool
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.schema import AgentAction, AgentFinish
import re

# init the llm
llm = OpenAI(openai_api_key="your-openai-api-key", model="gpt-3.5-turbo", temperature=0)

# defining tools
tools = [
    Tool(
        name="SearchAPI",
        func=lambda x: "Searching...",
        description="A tool to search the internet for information.",
    ),
]

# template for agent's behavior and decision-making process
template = """You are a helpful assistant with access to the following tools:

{tools}

Utilize these tools to assist with various tasks. Keep a record of your actions in the scratchpad.

{agent_scratchpad}

Please respond to the following input: {input}

Thought: Let's approach this systematically:
1) First, I should introduce myself politely.
2) Then, I'll address the user's specific question or request.

Action: None
Action Input: N/A

Response: Greetings! I'm an AI assistant developed by Anthropic. It's a pleasure to meet you. How may I be of assistance today?"""

prompt = PromptTemplate.from_template(template)

# custom output parser
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output):
        """Analyzing LLM output"""
        match = re.search(r"Action: (.*?)\nObservation: (.*?)(\n|$)", llm_output, re.DOTALL)
        if match:
            return AgentAction(tool=match.group(1).strip(), tool_input=match.group(2).strip())
        else:
            return AgentFinish(return_values={"output": llm_output.strip()}, log=llm_output)

output_parser = CustomOutputParser()

# creating LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# initializing LLMSingleActionAgent
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=[tool.name for tool in tools]
)

# setting up the AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools)

# preparing input data for the agent
input_data = {
    "input": "I'm Mihika, who are you?",
    "agent_scratchpad": "",
    "tools": "\n".join([f"- {tool.name}: {tool.description}" for tool in tools])
}

# executing agent
response = agent_executor.run(input_data)
print(response)